## PROJE
##### Konu: imdb veri setini pyspark ile analiz etmek.

In [1]:
import findspark 
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
findspark.init()
findspark.find()

'/opt/conda/lib/python3.10/site-packages/pyspark'

In [2]:
csv_path = '/workspaces/codespaces-jupyter/data/movie_metadata.csv'
try:
    sc = SparkContext()
except ValueError:
    sc.stop()       #type:ignore
    sc = SparkContext()
finally:
    rdd = sc.textFile(csv_path)     #type:ignore
    
spark = SparkSession.builder.appName("Film Analizi").getOrCreate()
films_data = spark.read.options(header="true", inferSchema="true") \
             .csv(csv_path)

23/04/17 12:30:46 WARN Utils: Your hostname, codespaces-282530 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/04/17 12:30:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/17 12:30:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.functions import desc


films = films_data.filter(films_data.budget.isNotNull()).sort(desc('imdb_score'))
# datayı temizleyelim ve ardından sorularda işimizi kolaylaştırmak için bütün listeyi imdb puanına göre sıralayalım

#temizlemek istediğimiz data olursa göndermemiz için yazdık
def data_cleaning(data):
    data = data.filter(films.budget.isNotNull())
    return data

In [4]:
films.show(30) # yukarıda yaptığımız işlemi verimizin sadece ilk 30 satırında test edelim

23/04/17 12:31:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+----------------+--------------------+----------------------+--------+-----------------------+----------------------+--------------------+----------------------+---------+--------------------+-------------------+--------------------+---------------+-------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------------+---------+----------+----------------------+----------+------------+--------------------+
|           color|       director_name|num_critic_for_reviews|duration|director_facebook_likes|actor_3_facebook_likes|        actor_2_name|actor_1_facebook_likes|    gross|              genres|       actor_1_name|         movie_title|num_voted_users|cast_total_facebook_likes|        actor_3_name|facenumber_in_

## <br>Analiz Soruları
<br>
<br>

1- Hangi ülkenin kaç tane filmi ilk 5 bin arasında yer alıyor?


In [6]:
# ülkeleri imdb puanlarına göre sıralayalım ve hangi ülkelerin ilk 5 bine girdiğini hesaplayalım
# sonra ülke bazında gruplama yapıp film sayılarını hesaplayalım
from pyspark.sql.functions import desc, count, col

top_5000_films = films.orderBy(desc("imdb_score")).limit(5000)

film_counts_by_country = (top_5000_films.groupBy("country")
                          .agg(count(col("*")).alias("film_count"))
                          .orderBy(desc("film_count"))
                          .show(30))


+--------------+----------+
|       country|film_count|
+--------------+----------+
|           USA|      3509|
|            UK|       379|
|        France|       130|
|        Canada|       109|
|       Germany|        90|
|     Australia|        50|
|         Spain|        27|
|         China|        23|
|         Italy|        20|
|         Japan|        20|
|         India|        19|
|     Hong Kong|        15|
|        Mexico|        14|
|   New Zealand|        13|
|   South Korea|        12|
|        Russia|        10|
|       Denmark|         9|
|  South Africa|         8|
|       Ireland|         8|
|        Norway|         8|
|        Brazil|         7|
|   Netherlands|         5|
|     Argentina|         4|
|       Romania|         4|
|          Iran|         4|
|      Thailand|         4|
|        Sweden|         3|
|        Israel|         3|
|Czech Republic|         3|
|   Switzerland|         3|
+--------------+----------+
only showing top 30 rows



2- Her ülkenin ortalama imdb puanı kaç?


In [7]:
from pyspark.sql.functions import avg,desc,round
imdb_country_avg = (films.groupBy("country")
                    .agg(round(avg("imdb_score"),1).alias("imdb ortalamasi"))
                    .orderBy(desc("imdb ortalamasi")).show(30,truncate=False))

+--------------------+---------------+
|country             |imdb ortalamasi|
+--------------------+---------------+
|Kyrgyzstan          |8.7            |
|Libya               |8.4            |
|United Arab Emirates|8.2            |
|Egypt               |8.1            |
|Soviet Union        |8.1            |
|West Germany        |7.9            |
|Iran                |7.7            |
|Indonesia           |7.6            |
|Argentina           |7.5            |
|Israel              |7.5            |
|Colombia            |7.5            |
|Sweden              |7.5            |
|Afghanistan         |7.4            |
|Brazil              |7.4            |
|New Zealand         |7.4            |
|Kenya               |7.4            |
|Denmark             |7.2            |
|Panama              |7.2            |
|Taiwan              |7.2            |
|Finland             |7.2            |
|Czech Republic      |7.0            |
|Pakistan            |7.0            |
|Ireland             |7.0

3- Her yönetmenin ortalama imdb puanı kaç?


In [8]:
director_groups = films.groupBy("director_name")

director_avg_scores = (director_groups
                       .agg(round(avg("imdb_score"),1)
                            .alias("avg_score")))
director_avg_scores_sorted = director_avg_scores.sort("avg_score", ascending=False).show(30,truncate=False)

+---------------------+---------+
|director_name        |avg_score|
+---------------------+---------+
|Cary Bell            |8.7      |
|Sadyk Sher-Niyaz     |8.7      |
|Mike Mayhall         |8.6      |
|Charles Chaplin      |8.6      |
|Sergio Leone         |8.5      |
|Majid Majidi         |8.5      |
|Ron Fricke           |8.5      |
|Damien Chazelle      |8.5      |
|Raja Menon           |8.5      |
|S.S. Rajamouli       |8.4      |
|Robert Mulligan      |8.4      |
|Christopher Nolan    |8.4      |
|Marius A. Markevicius|8.4      |
|Jay Oliva            |8.4      |
|Asghar Farhadi       |8.4      |
|Richard Marquand     |8.4      |
|Bill Melendez        |8.4      |
|Moustapha Akkad      |8.4      |
|Stanley Donen        |8.3      |
|Lee Unkrich          |8.3      |
|Lenny Abrahamson     |8.3      |
|John Sturges         |8.3      |
|Fritz Lang           |8.3      |
|Sut Jhally           |8.3      |
|Joe Kenemore         |8.2      |
|Rajkumar Hirani      |8.2      |
|Hayao Miyazak

4- En çok hangi türde film ilk 5000'de yer almıştır


In [9]:
# 1.Yol

from pyspark.sql.functions import desc

# Sütunları seç
movies = films.select("genres", "imdb_score")

# Sadece ilk 5000 filmden oluşan bir veri kümesi oluştur
movies = movies.orderBy(desc("imdb_score")).limit(5000)

# '|' karakteriyle ayrılmış türlerin listesini al
genres = movies.rdd.flatMap(lambda row: row[0].split("|"))

# Her türün sayısını say ve en yüksek olanı al
top_genre = genres.countByValue().items()
top_genre = sorted(top_genre, key=lambda x: x[1], reverse=True)[0][0]

# Cevabı yazdır
print("En çok {} türündeki filmler ilk 5 binde yer almaktadır.".format(top_genre))

En çok Drama türündeki filmler ilk 5 binde yer almaktadır.


In [10]:
# 2.Yol

from pyspark.sql.functions import desc, split

movies_filtered = films.filter(
    films.gross.isNotNull())

movies = movies_filtered.select("genres", "imdb_score")

top_movies = movies.orderBy(desc("imdb_score")).limit(5000)

top_genres = top_movies.select(
    split(top_movies.genres, "\|")
    .alias("genre")).selectExpr("explode(genre) as genre")

(top_genres.groupBy("genre")
 .count()
 .sort(desc("count"))
 .show(1, False))

+-----+-----+
|genre|count|
+-----+-----+
|Drama|1961 |
+-----+-----+
only showing top 1 row



In [11]:
# 3.Yol

from pyspark.sql.functions import explode, split

df_films = films.withColumn("genre", explode(split("genres","\|")))
category_count = (df_films
                  .withColumn("genre", 
                              explode(split("genres", "\|")))
                  .groupBy("genre").count())
category_count = category_count.sort("count",ascending=False)
category_count.show(30,truncate=False) # bizim ihtiyacımız olan en çok hangi tür olduğudur
                        # bu yüzden show içerisine kaç yazdığımızın bir önemi yok

+-----------+-----+
|genre      |count|
+-----------+-----+
|Drama      |6841 |
|Comedy     |4998 |
|Thriller   |4528 |
|Action     |4047 |
|Adventure  |3464 |
|Romance    |3218 |
|Crime      |2793 |
|Fantasy    |2241 |
|Sci-Fi     |2123 |
|Family     |2079 |
|Mystery    |1622 |
|Horror     |1474 |
|Animation  |1059 |
|Biography  |941  |
|War        |736  |
|History    |734  |
|Music      |613  |
|Musical    |507  |
|Sport      |506  |
|Western    |303  |
|Documentary|179  |
|Film-Noir  |18   |
|Short      |10   |
|News       |6    |
|Reality-TV |3    |
|Game-Show  |3    |
+-----------+-----+



5- En fazla bütçeye sahip olan filmler toplamda ne kadar hasılat elde etti? En fazla bütçeye sahip olan ilk 20 filmi karşılaştırın ortalama hasılatı yazdırın.


In [12]:
from pyspark.sql.functions import desc


budget_revenue = (films.select('budget', 'gross', 'movie_title')
                  .orderBy(desc('budget'))
                  .head(20))
budged_filtered_20 = films.select("movie_title","budget").orderBy("budget",ascending=False)

total_revenue = sum([film.gross for film in budget_revenue])

average_revenue = total_revenue/20

print("En yüksek bütçeye sahip olan ilk 20 film, toplamda {} hasılat elde etti. Ortalama hasılat ise {}.".format(total_revenue, average_revenue))
budged_filtered_20.show(30,truncate=False)

En yüksek bütçeye sahip olan ilk 20 film, toplamda 1535068309 hasılat elde etti. Ortalama hasılat ise 76753415.45.
+--------------------------------------------+-----------+
|movie_title                                 |budget     |
+--------------------------------------------+-----------+
|The Host                                    |12215500000|
|Lady Vengeance                              |4200000000 |
|Fateless                                    |2500000000 |
|Princess Mononoke                           |2400000000 |
|Steamboy                                    |2127519898 |
|Akira                                       |1100000000 |
|Godzilla 2000                               |1000000000 |
|Kabhi Alvida Naa Kehna                      |700000000  |
|Tango                                       |700000000  |
|Kites                                       |600000000  |
|Red Cliff                                   |553632000  |
|The Legend of Suriyothai                    |400000000  |


6- En çok beğeni almış siyah beyaz filmlerin isimlerini, çıkış tarihlerini ve yönetmenlerini yazdırın.

In [13]:
blacknwhite_movies = (films.filter(films.color == " Black and White")
                         .select("movie_title", "title_year","director_name")
                         .orderBy(films.num_user_for_reviews.desc())) 
blacknwhite_movies.show(30,truncate=False)


+---------------------------------------------------------------------+----------+------------------+
|movie_title                                                          |title_year|director_name     |
+---------------------------------------------------------------------+----------+------------------+
|The Blair Witch Project                                              |1999      |Daniel Myrick     |
|Moulin Rouge!                                                        |2001      |Baz Luhrmann      |
|Casino Royale                                                        |2006      |Martin Campbell   |
|Casino Royale                                                        |2006      |Martin Campbell   |
|Kill Bill: Vol. 1                                                    |2003      |Quentin Tarantino |
|Memento                                                              |2000      |Christopher Nolan |
|The Phantom of the Opera                                             |2004      |

7- Drama, Comedy, Romance, War, Sci-Fi kategorilerinde filmleri seven bir kullanıcıya film önerisi yapın.

In [14]:
from pyspark.sql.functions import col

# kullanıcın istediğ kategorileri bir dizi haline getirelim ki sonradan değiştirmemiz kolay olsun
selected_genres = ["Drama", "Comedy", "Romance", "War", "Sci-Fi"]

selected_movies = films.filter(col("genres").isin(selected_genres))

# imdb_score sütununa göre sıralayalım
sorted_movies = selected_movies.orderBy(col("imdb_score").desc())

# En yüksek puan alan 10 filmi öneri olarak verelim
top_movies = sorted_movies.select("movie_title", "imdb_score").limit(10)

# Öneri listesini gösterelim
top_movies.show(30,truncate=False)


+---------------------------------------------------------------------+----------+
|movie_title                                                          |imdb_score|
+---------------------------------------------------------------------+----------+
|Fight Club                                                           |8.8       |
|One Flew Over the Cuckoo's Nest                                      |8.7       |
|Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb |8.5       |
|American Beauty                                                      |8.4       |
|Requiem for a Dream                                                  |8.4       |
|Room                                                                 |8.3       |
|Good Will Hunting                                                    |8.3       |
|The Hunt                                                             |8.3       |
|Trainspotting                                                        |8.2       |
|Gra

8- Quentin Tarantino'nun ve en çok beğeni toplamış 5 filmini kategorileriyle, çıkış yılı ve isimleriyle birlikte yazdırın.

In [15]:
from pyspark.sql.functions import desc

tarantino_movies = films.filter(films.director_name == "Quentin Tarantino" ) 
sorted_movies = tarantino_movies.sort(desc("imdb_score"))

top_movies = sorted_movies.limit(5)
top_movies.select("genres", "title_year", "movie_title", "imdb_score").show(truncate=False)

+--------------------+----------+---------------------+----------+
|genres              |title_year|movie_title          |imdb_score|
+--------------------+----------+---------------------+----------+
|Crime|Drama         |1994      |Pulp Fiction         |8.9       |
|Drama|Western       |2012      |Django Unchained     |8.5       |
|Crime|Drama|Thriller|1992      |Reservoir Dogs       |8.4       |
|Adventure|Drama|War |2009      |Inglourious Basterds |8.3       |
|Action              |2003      |Kill Bill: Vol. 1    |8.1       |
+--------------------+----------+---------------------+----------+



9- Kullanıcının seçtiği (3)kategorileri bir sözlük yapısında tutun. Kullanıcıya bu sözlük yapısında bulunan kategorilerin her birinden 10 adet film önerisinde bulunun.

In [16]:
# Kullanıcının seçtiği kategorileri belirleyelim
kategoriler = ["Drama", "Comedy", "Romance"]

# Sözlük yapısı oluşturalım
kategoriler_dict = {}
for kategori in kategoriler:
    kategoriler_dict[kategori] = films.filter(films.genres.contains(kategori)).limit(10)

print("Drama kategorisindeki öneriler:")
kategoriler_dict["Drama"].select("movie_title", "genres").show(truncate=False)

print("Comedy kategorisindeki öneriler:")
kategoriler_dict["Comedy"].select("movie_title", "genres").show(truncate=False)

print("Romance kategorisindeki öneriler:")
kategoriler_dict["Romance"].select("movie_title", "genres").show(truncate=False)




Drama kategorisindeki öneriler:
+--------------------------------------------------+------------------------------+
|movie_title                                       |genres                        |
+--------------------------------------------------+------------------------------+
|The Shawshank Redemption                          |Crime|Drama                   |
|The Godfather                                     |Crime|Drama                   |
|The Dark Knight                                   |Action|Crime|Drama|Thriller   |
|The Godfather: Part II                            |Crime|Drama                   |
|The Lord of the Rings: The Return of the King     |Action|Adventure|Drama|Fantasy|
|Schindler's List                                  |Biography|Drama|History       |
|Pulp Fiction                                      |Crime|Drama                   |
|12 Angry Men                                      |Crime|Drama                   |
|Forrest Gump                               

10- Kullanıcının izlediği diğer filmlere dayanarak bir kullanıcı için film önerileri yapın. Kullanıcının izlediği filmler bir sözlük halinde bulunsun.

In [19]:
# Aşağıda yazmış olduğumuz kod, 
# kullanıcının izlediği filmlerin türlerine göre diğer filmleri filtreleyerek 
# ve toplam Facebook beğeni sayısına ve IMDB puanına göre sıralayarak, 
# kullanıcıya önerilerde bulunur. 
# Önerilen filmler recommended_movies listesi içinde depolanır 
# ve daha sonra döngü ile yazdırılır.

In [17]:
from pyspark.sql.functions import col

# kullanıcının izlediği filmler
user_watched_movies = {'Avatar ', 'The Dark Knight ', 'Inception ', 'The Matrix ', 'Interstellar '}

# kullanıcının izlediği filmler filtrelenir ve filmlerin türleri explode edelim
user_watched_movies_genres = films.filter(col('movie_title').isin(user_watched_movies)) \
    .select('movie_title', 'genres') \
    .withColumn('genres', explode(split(col('genres'), '\|'))) \
    .select('genres')

# kullanıcının izlediği filmlerin türlerine göre öneri filmler seçelim
recommended_movies = films.filter(col('genres').isin(user_watched_movies_genres.rdd.map(lambda x: x[0]).collect())) \
    .filter(~col('movie_title').isin(user_watched_movies)) \
    .groupBy('movie_title') \
    .agg({'movie_facebook_likes': 'sum', 'imdb_score': 'max'}) \
    .withColumnRenamed('sum(movie_facebook_likes)', 'total_facebook_likes') \
    .withColumnRenamed('max(imdb_score)', 'imdb_score') \
    .sort(col('total_facebook_likes').desc(), col('imdb_score').desc()) \
    .limit(10) \
    .select('movie_title', 'total_facebook_likes', 'imdb_score') \
    .collect()

# öneri filmleri yazdıralım
print("Recommended movies based on user's watched movies:")
for movie in recommended_movies:
    print(movie['movie_title'], '- Total Facebook likes:', movie['total_facebook_likes'], '- IMDB Score:', movie['imdb_score'])


Recommended movies based on user's watched movies:
Boyhood  - Total Facebook likes: 92000 - IMDB Score: 8.0
The Help  - Total Facebook likes: 75000 - IMDB Score: 8.1
Room  - Total Facebook likes: 72000 - IMDB Score: 8.3
The Hunt  - Total Facebook likes: 60000 - IMDB Score: 8.3
Fight Club  - Total Facebook likes: 48000 - IMDB Score: 8.8
Still Alice  - Total Facebook likes: 45000 - IMDB Score: 7.5
Requiem for a Dream  - Total Facebook likes: 38000 - IMDB Score: 8.4
Blue Jasmine  - Total Facebook likes: 36000 - IMDB Score: 7.3
One Flew Over the Cuckoo's Nest  - Total Facebook likes: 32000 - IMDB Score: 8.7
Good Will Hunting  - Total Facebook likes: 31000 - IMDB Score: 8.3
